In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# импортируем классы BaseEstimator и TransformerMixin,
# позволяющие написать собственные классы
from sklearn.base import BaseEstimator, TransformerMixin
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс Pipeline, позволяющий создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс GridSearchCV, позволяющий 
# выполнить решетчатый поиск
from sklearn.model_selection import GridSearchCV
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем класс PowerTransformer, позволяющий выполнить 
# преобразование Бокса-Кокса/Йео-Джонсона и стандартизацию
from sklearn.preprocessing import PowerTransformer
# импортируем класс OneHotEncoder, позволяющий
# выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс KFold и функцию cross_val_score
# для выполнения перекрестной проверки
from sklearn.model_selection import KFold, cross_val_score
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Модуль_1/Data/Verizon_miss_dubl.csv', sep=';')

In [6]:
# смотрим первые 5 наблюдений
data.head()

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,27.09,0.0,39.74,Нет,Бюджетный,CC,35.0,Женский,Женат,0.0,77680.0,0
1,NaN,0.0,46.31,Нет,NaN,NaN,53.0,Мужской,Одинокий,1.0,37111.5,0
2,23.76,0.0,NaN,NaN,Бюджетный,Auto,NaN,Женский,NaN,1.0,37079.4,0
3,9.40,NaN,13.90,Нет,NaN,CH,NaN,Мужской,Одинокий,NaN,81997.0,0
4,14.15,0.0,108.43,Да,Бесплатный,Auto,39.0,Женский,Одинокий,0.0,16829.6,0


In [7]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Data columns (total 12 columns):
longdist    4430 non-null float64
internat    4427 non-null float64
local       4428 non-null float64
int_disc    4430 non-null object
billtype    4427 non-null object
pay         4429 non-null object
age         4428 non-null float64
gender      4430 non-null object
marital     4427 non-null object
children    4430 non-null float64
income      4430 non-null float64
churn       4431 non-null int64
dtypes: float64(6), int64(1), object(5)
memory usage: 415.5+ KB


In [5]:
# заменяем запятые на точки и преобразуем в тип float
for i in ['longdist', 'internat', 'local', 'income']:
    data[i] = data[i].str.replace(',', '.').astype('float')

In [8]:
# создаем список категориальных переменных
cat_cols = data.select_dtypes(include=['object']).columns.tolist()
# смотрим уникальные значения этих переменных
for i in cat_cols:
    print(i, data[i].unique())

int_disc ['Нет' nan 'Да']
billtype ['Бюджетный' nan 'Бесплатный']
pay ['CC' nan 'Auto' 'CH' 'CD']
gender ['Женский' 'Мужской' nan 'Женский&*' 'Мужской&*']
marital ['Женат' 'Одинокий' nan '_Одинокий' '_Женат' 'Же&нат']


In [9]:
# удаляем лишние символы в категориях переменных
# gender и marital
for i in ['gender', 'marital']:
    data[i] = data[i].str.replace('[*&_]', '')
    
# проверяем
for i in ['gender', 'marital']:
    print(i, data[i].unique())

gender ['Женский' 'Мужской' nan]
marital ['Женат' 'Одинокий' nan]


In [10]:
# смотрим частоты по категориальным переменным, 
# чтобы выявить редкие категории
for i in cat_cols:
    print(i)
    print('')
    print(data[i].value_counts(dropna=False))

int_disc

Нет    3054
Да     1376
NaN       1
Name: int_disc, dtype: int64
billtype

Бюджетный     2244
Бесплатный    2183
NaN              4
Name: billtype, dtype: int64
pay

CC      2561
CH       977
Auto     889
CD         2
NaN        2
Name: pay, dtype: int64
gender

Женский    2244
Мужской    2186
NaN           1
Name: gender, dtype: int64
marital

Женат       2625
Одинокий    1802
NaN            4
Name: marital, dtype: int64


In [11]:
# удаляем дубли на месте, оставляя первое
# встретившееся наблюдение в паттерне дубля
data.drop_duplicates(subset=None, keep='first', 
                     inplace=True)

In [12]:
# заменяем редкую категорию модой
data.at[data['pay'] == 'CD', 'pay'] = 'CC'

In [13]:
# создаем переменную - результат конъюнкции
data['gender_marital'] = data.apply(
    lambda x: f"{x['gender']} + {x['marital']}", 
    axis=1)

In [14]:
# поделим возраст на длительность междугородних звонков в минутах
data['ratio'] = data['age'] / data['longdist']
# заменяем бесконечные значения на 1
data['ratio'].replace([np.inf, -np.inf], 1, inplace=True)

In [15]:
# поделим длительность междугородних звонков в минутах на
# длительность международных звонков в минутах
data['ratio2'] = data['longdist'] / data['internat']
# заменяем бесконечные значения на 0
data['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)

In [16]:
# поделим доход на возраст
data['ratio3'] = data['income'] / data['age']
# заменяем бесконечные значения на 0
data['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)

In [17]:
# поделим возраст на количество детей
data['ratio4'] = data['age'] / data['children']
# заменяем бесконечные значения на 0
data['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

In [18]:
# разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data.drop('churn', axis=1), 
                                                    data['churn'],
                                                    test_size=0.3,
                                                    stratify=data['churn'],
                                                    random_state=42)

In [19]:
# создаем список категориальных переменных, список количественных 
# переменных, не предназначенных для биннинга, список 
# количественных переменных, предназначенных для биннинга
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index
num_columns_for_bin = ['age']

In [20]:
# создаем собственный класс, выполняющий биннинг
class CustomBinning(BaseEstimator, TransformerMixin):
    """
    Parameters:
        bins: список бинов.
    """
    def __init__(self, bins=[-np.inf,  30, 50, np.inf]):
        self.bins = bins
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        bins = np.array(self.bins)
        X_bin = np.digitize(X, bins)
        return X_bin

In [21]:
# создаем собственный класс, заменяющий отрицательные
# и нулевые значения на небольшие положительные
class Replacer(BaseEstimator, TransformerMixin):
    """
    Parameters:
        repl: значение для замены.
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X <= 0, self.repl_value, X)
        return X_replaced

In [22]:
# создаем трансформер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [23]:
# создаем трансформер для количественных переменных, 
# не предназначенных для биннинга
num_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('replacer', Replacer(repl_value=0.1)),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True))
])

In [24]:
# создаем трансформер для количественных переменных, 
# предназначенных для биннинга
num_bin_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('bin', CustomBinning()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [25]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название трансформера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('num_bin', num_bin_pipe, num_columns_for_bin),
                ('cat', cat_pipe, cat_columns)]

# передаем список в ColumnTransformer
ct = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
ml_pipe = Pipeline([('transform', ct), 
                    ('logreg', LogisticRegression(solver='liblinear'))])

In [26]:
# задаем сетку гиперпараметров
param_grid = {
    'transform__num__imputer__strategy': ['mean', 'median', 'constant'],
    'transform__num__replacer__repl_value': [0.1, 0.2, 0.3, 0.4, 0.5],
    'transform__cat__imputer__strategy': ['most_frequent', 'constant'],
    'transform__num_bin__imputer__strategy': ['mean', 'median'],
    'transform__num_bin__bin__bins': [[-np.inf, 30, 50, np.inf], 
                                      [-np.inf, 25, 45, np.inf]],
    'logreg__C': [.01, .1, .5, 1, 5, 10]
}
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs = GridSearchCV(ml_pipe, param_grid, cv=5, return_train_score=False)
# выполняем решетчатый поиск
gs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(
    gs.best_params_))
# смотрим наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(
    gs.best_score_))
# смотрим правильность на тестовой выборке
print('Значение правильности на тестовой выборке: {:.3f}'.format(
    gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'logreg__C': 1, 'transform__cat__imputer__strategy': 'constant', 'transform__num__imputer__strategy': 'median', 'transform__num__replacer__repl_value': 0.3, 'transform__num_bin__bin__bins': [-inf, 25, 45, inf], 'transform__num_bin__imputer__strategy': 'mean'}
Наилучшее значение правильности: 0.821
Значение правильности на тестовой выборке: 0.810


In [27]:
# записываем CSV-файл в объект DataFrame 
fulldata = pd.read_csv('Модуль_1/Data/Verizon_miss_dubl.csv', sep=';')

In [28]:
# пишем функцию, выполняющую предварительную 
# обработку всех исторических данных
def preprocessing(df):
    # заменяем запятые на точки и преобразуем в тип float
    for i in ['longdist', 'internat', 'local', 'income']:
        df[i] = df[i].str.replace(',', '.').astype('float')
    # удаляем возможные лишние символы (все символы, не являющиеся 
    # буквами, символы нижнего подчеркивания и цифры) в категориях 
    # переменных gender и marital
    for i in ['gender', 'marital']:
        df[i] = df[i].str.replace('[\d+\W_]', '')
    # удаляем дубли на месте, оставляя первое
    # встретившееся наблюдение в паттерне дубля
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    # все новые категории переменной pay заменяем модой
    lst = ['CC', 'Auto', 'CH', np.NaN]
    replace_new_values = lambda x: 'CC' if x not in lst else x
    df['pay'] = df['pay'].map(replace_new_values)
    # создаем переменную - результат конъюнкции
    df['gender_marital'] = df.apply(
        lambda x: f"{x['gender']} + {x['marital']}", 
        axis=1)                    
    # поделим возраст на длительность междугородних звонков в минутах
    df['ratio'] = df['age'] / df['longdist']
    # заменяем бесконечные значения на 1
    df['ratio'].replace([np.inf, -np.inf], 1, inplace=True)
    # поделим длительность междугородних звонков в минутах на
    # длительность международных звонков в минутах
    df['ratio2'] = df['longdist'] / df['internat']
    # заменяем бесконечные значения на 0
    df['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)
    # поделим доход на возраст
    df['ratio3'] = df['income'] / df['age']
    # заменяем бесконечные значения на 0
    df['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)
    # поделим возраст на количество детей
    df['ratio4'] = df['age'] / df['children']
    # заменяем бесконечные значения на 0
    df['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

In [29]:
# применяем функцию предварительной обработки 
# ко всем историческим данным
preprocessing(fulldata)

In [30]:
# создаем массив меток и массив признаков
y_fulldata = fulldata.pop('churn')

In [31]:
# модифицируем трансформеры с учетом найденных 
# оптимальных значений гиперпараметров
cat_pipe_full = Pipeline([
    ('imputer', SimpleImputer(strategy='constant')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

num_pipe_full = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('replacer', Replacer(repl_value=0.3)),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True))
])

num_bin_pipe_full = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('bin', CustomBinning(bins=[-np.inf, 25, 45, np.inf])),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [32]:
# создаем список модифицированных трансформеров
transformers_full = [('num', num_pipe_full, num_columns),
                     ('num_bin', num_bin_pipe_full, num_columns_for_bin),
                     ('cat', cat_pipe_full, cat_columns)]

# передаем список в ColumnTransformer
ct_full = ColumnTransformer(transformers=transformers_full)

In [33]:
# задаем итоговый конвейер с оптимальными 
# значениями гиперпараметров
ml_pipe_full = Pipeline([('transform', ct_full), 
                         ('logreg', LogisticRegression(solver='liblinear', C=0.5))])

In [34]:
# обучаем итоговый конвейер с оптимальными значениями 
# гиперпараметров на всех исторических данных
ml_pipe_full.fit(fulldata, y_fulldata)
# печатаем значение правильности
print('Правильность на всей исторической выборке: {:.3f}'.format(
    ml_pipe_full.score(fulldata, y_fulldata)))

Правильность на всей исторической выборке: 0.826


In [35]:
# записываем CSV-файл, содержащий новые данные,
# в объект DataFrame
newdata = pd.read_csv('Модуль_1/Data/Verizon_new.csv', 
                      encoding='utf-8', sep=';')

In [36]:
# применяем функцию предварительной обработки 
# к новым данным
preprocessing(newdata)

In [37]:
# при помощью итогового конвейера с оптимальными значениями 
# гиперпараметров, обученного на всей исторической выборке, 
# вычисляем вероятности для новых данных
prob = ml_pipe_full.predict_proba(newdata)
# выведем вероятности для первых 5 наблюдений
prob[:5]

array([[0.05482982, 0.94517018],
       [0.07168819, 0.92831181],
       [0.89441637, 0.10558363],
       [0.96067819, 0.03932181],
       [0.06889734, 0.93110266]])